In [6]:
from typing import cast
from itertools import combinations
import numpy as np
import scipy.optimize as opt
from pyscf import gto
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider

In [7]:
from networkx import identified_nodes
from sympy import symbols, Matrix, cos, sin, init_printing, pi, exp, I
from sympy.tensor.array import tensorproduct
from sympy.physics.quantum import Dagger

init_printing(use_unicode=True)

theta, phi = symbols('theta phi')
Z = Matrix([[1, 0], [0, -1]])
Y = Matrix([[0, -1j], [1j, 0]])

Ry = Matrix([
    [cos(theta/2), -sin(theta/2)],
    [sin(theta/2), cos(theta/2)]
])

Ry_dagger = Matrix([
    [cos(theta/2), sin(theta/2)],
    [-sin(theta/2), cos(theta/2)]
])

Rz = Matrix([
    [exp(-I * phi / 2), 0],
    [0, exp(I * phi / 2)]
])

Rz_dagger = Matrix([
    [exp(I * phi / 2), 0],
    [0, exp(-I * phi / 2)]
])

Identity = Matrix([
    [1, 0],
    [0, 1]
])

RzRy = Rz * Ry
RzRy_dagger = Ry_dagger * Rz_dagger

CNOT = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

CNOT2 = Matrix([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0]
])

def tensor(A, B):
    C = tensorproduct(A, B)

    D = Matrix([
        [C[0][0][0][0], C[0][0][0][1], C[0][1][0][0], C[0][1][0][1]],
        [C[0][0][1][0], C[0][0][1][1], C[0][1][1][0], C[0][1][1][1]],
        [C[1][0][0][0], C[1][0][0][1], C[1][1][0][0], C[1][1][0][1]],
        [C[1][0][1][0], C[1][0][1][1], C[1][1][1][0], C[1][1][1][1]]
    ])

    return D

Rgate = tensor(Identity, RzRy)
Rgate_dagger = tensor(Identity, RzRy_dagger)


M1 = Rgate_dagger  * CNOT2

M2 = CNOT * M1

M3 = Rgate * M2

M4 = CNOT2 * M3

M4



⎡   2⎛θ⎞      2⎛θ⎞                                                            
⎢sin ⎜─⎟ + cos ⎜─⎟                 0                               0          
⎢    ⎝2⎠       ⎝2⎠                                                            
⎢                                                                             
⎢                                ⎛θ⎞    ⎛θ⎞             ⅈ⋅φ    2⎛θ⎞    ⅈ⋅φ    
⎢        0                  2⋅sin⎜─⎟⋅cos⎜─⎟          - ℯ   ⋅sin ⎜─⎟ + ℯ   ⋅cos
⎢                                ⎝2⎠    ⎝2⎠                     ⎝2⎠           
⎢                                                                             
⎢                      -ⅈ⋅φ    2⎛θ⎞    -ⅈ⋅φ    2⎛θ⎞              ⎛θ⎞    ⎛θ⎞   
⎢        0          - ℯ    ⋅sin ⎜─⎟ + ℯ    ⋅cos ⎜─⎟        -2⋅sin⎜─⎟⋅cos⎜─⎟   
⎢                               ⎝2⎠             ⎝2⎠              ⎝2⎠    ⎝2⎠   
⎢                                                                             
⎢                                                   

In [8]:
from networkx import identified_nodes
from numpy import disp
from sympy import symbols, Matrix, cos, sin, init_printing, pi, exp, I,  Rational, sqrt, symbols
from sympy import UnevaluatedExpr, latex
from IPython.display import Math
from sympy.tensor.array import tensorproduct
from sympy.physics.quantum import Dagger

q1 = Matrix([
    [1],
    [0]
])

q2 = Matrix([
    [0],
    [1]
])

two = symbols('2')

# 스칼라를 행렬 외부에 표시
scalar = 1/sqrt(two)

# 행렬 정의
Hgate_matrix = Matrix([
    [1, 1],
    [1, -1]
])

latex_q1 = latex(q1)
latex_q2 = latex(q2)
latax_sc = latex(scalar)
latax_H = latex(Hgate_matrix)

display_latex = f'{latax_sc}{latax_H}{latex_q1} = {latax_sc}{latex_q1} + {latax_sc}{latex_q2}'
display(Math(display_latex))

<IPython.core.display.Math object>

In [9]:
import time

# 딕셔너리 예시
my_dict = {
    'apple': 'red',
    'banana': 'yellow',
    'cherry': 'red'
}

# 각 원소를 개별적으로 출력
for key, value in my_dict.items():
    print(f"{key}: {value}")
    time.sleep(1)  # 잠시 대기

# 커서를 각 항목 수만큼 위로 이동
for _ in my_dict:
    print('\x1b[1A', end='')  # 커서 한 줄 위로 이동

# 같은 위치에서 다시 출력
for key, value in my_dict.items():
    print(f"{key}: {value}")


apple: red
banana: yellow
cherry: red
apple: red
banana: yellow
cherry: red


In [10]:
Z1 = tensor(Rz, Rz)

Y1 = tensor(Identity, Ry_dagger)
Y2 = tensor(Identity, Ry)

N1 = Y1 * CNOT2 * Z1
N2 = CNOT * N1
N3 = Y2 * N2
N4 = CNOT2 * N3
N4


⎡ -ⅈ⋅φ    2⎛θ⎞    -ⅈ⋅φ    2⎛θ⎞                                                
⎢ℯ    ⋅sin ⎜─⎟ + ℯ    ⋅cos ⎜─⎟           0                    0               
⎢          ⎝2⎠             ⎝2⎠                                                
⎢                                                                             
⎢                                      ⎛θ⎞    ⎛θ⎞         2⎛θ⎞      2⎛θ⎞      
⎢              0                  2⋅sin⎜─⎟⋅cos⎜─⎟    - sin ⎜─⎟ + cos ⎜─⎟      
⎢                                      ⎝2⎠    ⎝2⎠          ⎝2⎠       ⎝2⎠      
⎢                                                                             
⎢                                    2⎛θ⎞      2⎛θ⎞         ⎛θ⎞    ⎛θ⎞        
⎢              0                - sin ⎜─⎟ + cos ⎜─⎟   -2⋅sin⎜─⎟⋅cos⎜─⎟        
⎢                                     ⎝2⎠       ⎝2⎠         ⎝2⎠    ⎝2⎠        
⎢                                                                             
⎢                                                   

In [11]:

Ua = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, exp(-I * phi / 2)],
    [0, 0, exp(-I * phi / 2), 0]
])

Ub = Matrix([
    [1, 0, 0, 0],
    [0, cos(theta), 0, sin(theta)],
    [0, 0, 1, 0],
    [0, sin(theta), 0, -cos(theta)]
])

Uc = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, exp(I * phi / 2)],
    [0, 0, exp(I * phi / 2), 0]
])

U = Uc * Ub * Ua

U

⎡1       0             0        0⎤
⎢                                ⎥
⎢                 -ⅈ⋅φ           ⎥
⎢                 ─────          ⎥
⎢                   2            ⎥
⎢0    cos(θ)     ℯ     ⋅sin(θ)  0⎥
⎢                                ⎥
⎢    ⅈ⋅φ                         ⎥
⎢    ───                         ⎥
⎢     2                          ⎥
⎢0  ℯ   ⋅sin(θ)     -cos(θ)     0⎥
⎢                                ⎥
⎣0       0             0        1⎦

In [12]:

Ua = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

Ub = Matrix([
    [1, 0, 0, 0],
    [0, cos(theta), 0, I * sin(theta)],
    [0, 0, 1, 0],
    [0, -I * sin(theta), 0, cos(theta)]
])

Uc = Ua

U = Uc * Ub * Ua

U

⎡1      0         0      0⎤
⎢                         ⎥
⎢0   cos(θ)    ⅈ⋅sin(θ)  0⎥
⎢                         ⎥
⎢0  -ⅈ⋅sin(θ)   cos(θ)   0⎥
⎢                         ⎥
⎣0      0         0      1⎦

In [13]:
from re import M


P = Matrix([
    [1, 0],
    [0, exp(I * pi / 4)]
])

Ry = Matrix([
    [cos(theta/2), -sin(theta/2)],
    [sin(theta/2), cos(theta/2)]
])

Ry_m = Matrix([
    [cos(theta/2), sin(theta/2)],
    [-sin(theta/2), cos(theta/2)]
])

X = Matrix([
    [0, 1],
    [1, 0]
])

P_m = Matrix([
    [1, 0],
    [0, exp(-I * pi / 4)]
])

result = P_m * Ry_m * X * Ry * P_m

result

⎡                                                          -ⅈ⋅π ⎤
⎢                                                          ─────⎥
⎢              ⎛θ⎞    ⎛θ⎞           ⎛     2⎛θ⎞      2⎛θ⎞⎞    4  ⎥
⎢         2⋅sin⎜─⎟⋅cos⎜─⎟           ⎜- sin ⎜─⎟ + cos ⎜─⎟⎟⋅ℯ     ⎥
⎢              ⎝2⎠    ⎝2⎠           ⎝      ⎝2⎠       ⎝2⎠⎠       ⎥
⎢                                                               ⎥
⎢   -ⅈ⋅π             -ⅈ⋅π                                       ⎥
⎢   ─────            ─────                                      ⎥
⎢     4      2⎛θ⎞      4      2⎛θ⎞               ⎛θ⎞    ⎛θ⎞     ⎥
⎢- ℯ     ⋅sin ⎜─⎟ + ℯ     ⋅cos ⎜─⎟      -2⋅-ⅈ⋅sin⎜─⎟⋅cos⎜─⎟     ⎥
⎣             ⎝2⎠              ⎝2⎠               ⎝2⎠    ⎝2⎠     ⎦